# Using Langraph to create Agents

### Using REACT Architecture (Reasoning + Acting)

REACT Architecture

Thinks and then reacts and makes a decision about which action to do. The action is performed in an environment and it returns an observation. Then an observation is returned. With that observation the LLM then repeats and goes until the Agent decides when it is done.

In [8]:

import openai
import re
import httpx
import os
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI

_ = load_dotenv(find_dotenv('credentials.env'))

### Doing a easy example just to make sure that is working

In [ ]:
client = OpenAI()  

chat_completion = client.chat.completions.create(

    model = 'gpt-5-nano',
    messages = [{'role': 'user', 'content': 'Hello world'}]

)

chat_completion.choices[0].message.content

### Creating an Agent by a class

In [35]:
class Agent:

    def __init__(self, system = ""):
        self.system = system #This is the message from the system
        self.messages = [] #Here we're going to append the history of messages.
        if self.system:
            self.messages.append({'role': 'system', 'content':system})

    def __call__(self, message):
        self.messages.append({'role': 'user', 'content': message}) #Append to the existin array of messages
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result}) #Appending the output comming from the assistant.
        return result
    
    def execute(self):

        completion = client.chat.completions.create(

            model = "gpt-5-nano",
            messages = self.messages)
        
        return completion.choices[0].message.content
    
    def get_messages(self):
        """Return all messages"""
        return self.messages

### Designing a Prompt

In [36]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [37]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [ ]:
#Here we define our agent
abot = Agent(prompt) #Using the class Init

In [39]:
result = abot("how much does a toy poodle weigh?") #It uses the class call
print(result)

Thought: I will fetch the average weight for a Toy Poodle from the dog weight database.
Action: average_dog_weight: Toy Poodle
PAUSE


In [40]:
result = average_dog_weight('Toy Poodle')
result

'a toy poodles average weight is 7 lbs'

In [42]:
# A continuation of the prompt
next_prompt = "Observation: {}".format(result)
next_prompt

'Observation: a toy poodles average weight is 7 lbs'

In [43]:
abot(next_prompt)

'Answer: A toy poodle weighs 7 lbs'

In [44]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'how much does a 